In [7]:
import csv

already_seen = set()
already_seen.add("") # 空白だけのコメントを除去するために、予め追加しておく
source_table = []
with open('eng_pubcom.csv', 'r', encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        id = row[0]
        text = "".join(row[3:])
        if text in already_seen:
            continue
        already_seen.add(text)
        source_table.append((id, text))

header = source_table[0]
source_table = source_table[1:]
source_table[:10]

[('1', '当社は地球温暖化防止の方向性について賛同し、第7次エネルギー基本計画に賛同いたします。'),
 ('2', '「バランスのとれたＳ＋３Ｅの実現」を基本方針として検討、策定されている点について、賛同する。'),
 ('3',
  '今回示されたエネルギー基本計画案は、S＋3Eの原則のもと、わが国の国民生活、経済活動を支えるエネルギー政策で、資源に乏しい島国であるわが国の地政学的なリスクも現実的に捉えており、脱炭素と経済性・安定供給のバランスを目指した内容で、概ね評価できるものである。とくに製紙業界では製造工程で大量の蒸気エネルギーが不可欠であり、一足飛びで火力の燃料を全てカーボンニュートラルとはならない状況を踏まえたエネルギーミックスとなっていることに賛同する。'),
 ('4', '第７次エネルギー基本計画（案）の方向性に賛同します'),
 ('5',
  '国の示す「我が国が将来にわたって豊かな国として存続し、全ての国民が希望をもって暮らせる社会を実現するためには、エネルギー安定供給、経済成長、脱炭素を同時に実現していく必要がある」という認識ならびに目指す方向性に賛同する。'),
 ('6',
  '第7次エネルギー基本計画（案）では、ウクライナ侵攻や中東情勢の緊迫化等に端を発したエネルギー安全保障の要請の高まり、DX,GX等の進展に伴う電力需要増加の見通し等エネルギーを取り巻く情勢変化を正面から捉え、S+3Eの観点は堅持しつつも、第6次エネルギー基本計画において「環境適合」に過度に傾斜していた部分を、「安定供給」、「経済効率性」の重要性を改めて捉え直した現実的な計画とされたことは、産業界の危機感と軌を一にするものと評価する。'),
 ('7',
  '今般示された第7次エネルギー基本計画、地球温暖化対策計画、GX2040ビジョンは、エネルギー政策と、その裏腹の関係にある地球温暖化政策のみならず、我が国の成長戦略とその根幹をなす産業政策までも一体化し、これらの計画、ビジョンを統合的、整合的に推進する政府の強い意志を鮮明に示すものと理解している。これら計画、ビジョンに掲げられた一連の方針に基づく具体的政策を着実に推進頂きたい。'),
 ('9', '日本の未来を大切に考えるのであれば、この第７次エネルギー基本計画案は廃案にし、一から基本計画をやり直す

In [8]:
# モデルを読み込む
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)


In [9]:
# エンベデッドする
sentences = [row[1] for row in source_table]

embeddings = model.encode(sentences)
print(embeddings.shape)

(1006, 768)


In [49]:
# コサイン類似度でクラスタリングする
import sklearn.cluster
# embeddingsをコサイン距離でクラスタリングする
# パラメータは要調整
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
cluster_model = sklearn.cluster.AgglomerativeClustering(n_clusters=None, metric='cosine', linkage='complete', distance_threshold=0.80, compute_full_tree=True)
clusters = cluster_model.fit_predict(embeddings)
# クラスタの個数の確認
len(set(clusters))

25

In [50]:
# 各クラスタの中心に最も近いノードを抽出する
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd

# クラスタごとにデータをグループ化
cluster_data = {}
for i, cluster_id in enumerate(clusters):
    if cluster_id not in cluster_data:
        cluster_data[cluster_id] = []
    cluster_data[cluster_id].append(i)

# 各クラスタの中心点（センテンス）を見つける
central_sentences = []

for cluster_id, indices in cluster_data.items():
    # クラスタが1つの要素しか持たない場合は、その要素を中心点とする
    if len(indices) == 1:
        central_sentences.append({
            'cluster_id': -1,
            'center_idx': indices[0],
            'sentence': sentences[indices[0]],
            'size': 1
        })
        continue
    
    # クラスタ内のエンベディングを取得
    cluster_embeddings = embeddings[indices]
    
    # クラスタのセントロイド（平均ベクトル）を計算
    centroid = np.mean(cluster_embeddings, axis=0)
    
    # セントロイドに最も近い点を見つける（コサイン類似度を使用）
    min_dist = float('inf')
    center_idx = -1
    
    for i, idx in enumerate(indices):
        dist = cosine(centroid, embeddings[idx])
        if dist < min_dist:
            min_dist = dist
            center_idx = idx
    
    central_sentences.append({
        'cluster_id': cluster_id,
        'center_idx': center_idx,
        'sentence': sentences[center_idx],
        'size': len(indices)
    })

# 結果をデータフレームとして表示
central_df = pd.DataFrame(central_sentences)
# クラスタサイズでソート（大きい順）
central_df = central_df.sort_values('size', ascending=False)
central_df

,cluster_id,center_idx,sentence,size
9,0,521,原発は持続可能なエネルギー源として大きな問題があり、核廃棄物の処理・管理の困難性はもとより、...,204
0,22,842,既存火力への追設等を念頭に、脱炭素化を見据え、石炭ガス化複合発電(IGCC)等の次世代の高効...,167
3,4,805,再生可能エネルギーの優先接続・優先給電を求める。原発の出力は微調整できず使うほどコストが安く...,85
12,7,103,エネルギー安定供給と脱炭素の両立を目指している点は評価できる,73
18,2,923,意見内容燃料アンモニアを活用した火力発電の脱炭素化を、水素社会推進法に基づき、低炭素燃料アン...,62
7,24,349,化石燃料の使用を一刻も早くやめてほしい。地球温暖化で私の未来が不安にさらされるのは嫌だ。また...,54
8,1,422,原発に使う予算があるなら再エネに予算を振り向け、各家庭の屋根やビルの壁面などに太陽光パネルを...,47
4,5,347,原発の新規増設を含む第7次エネルギー基本計画に反対します。当面、最悪の事故後でも再生可能な化...,33
6,21,19,今回の計画案ではこの気候危機を止めるには不十分であるので、これからの暮らしや次世代のためにも...,29
11,23,948,日本は水素・アンモニア混焼やCCSの技術が排出削減対策に該当するとしていますが、火力発電は、...,29


In [ ]:
# 意見の数が多い順にcluster_idを振り直す
new_cluster_id = []
for i, cluster_id in enumerate(central_df['cluster_id']):
    new_cluster_id.append(cluster_id)

In [54]:
# id, cluster_id, distance to cluster center, sentence　というテーブルを作る
# cluster_sizeが1ならば、cluster_idは-1にする

result_table = []
for new_cluster_id, indices in cluster_data.items():
    for idx in indices:
        dist_to_center = cosine(embeddings[idx], embeddings[central_sentences[cluster_id]['center_idx']])
        result_table.append({
            'id': idx,
            'cluster_id': cluster_id,
            'distance_to_center': dist_to_center,
            'sentence': sentences[idx]
        })

result_df = pd.DataFrame(result_table)
result_df

,id,cluster_id,distance_to_center,sentence
0,0,24,0.610175,当社は地球温暖化防止の方向性について賛同し、第7次エネルギー基本計画に賛同いたします。
1,3,24,0.570002,第７次エネルギー基本計画（案）の方向性に賛同します
2,4,24,0.507703,国の示す「我が国が将来にわたって豊かな国として存続し、全ての国民が希望をもって暮らせる社会を...
3,6,24,0.617628,今般示された第7次エネルギー基本計画、地球温暖化対策計画、GX2040ビジョンは、エネルギー...
4,7,24,0.523677,日本の未来を大切に考えるのであれば、この第７次エネルギー基本計画案は廃案にし、一から基本計画...
...,...,...,...,...
1001,430,24,0.369750,ペロブスカイト太陽電池は紫外線や湿度などの外的要因の影響を受けやすく、劣化しやすい特性がある...
1002,432,24,0.304322,ペロブスカイト太陽電池に問題を言及している専門家もいるため、慎重に進めたほうが良い。
1003,433,24,0.432266,過疎化が進む農村部は食糧自給率を鑑みても営農型太陽光発電が良いと考えられる。
1004,434,24,0.537635,営農従事者の高齢化問題を考慮すれば、営農型太陽光の拡大は非常に大きなポテンシャルを秘めており...


In [55]:
# excelに出力、それぞれのクラスタごとにシートを切る、distance_to_centerが大きい順にソートする
import pandas as pd
from openpyxl import Workbook

# Excel writerを作成
with pd.ExcelWriter('clustering_results.xlsx', engine='openpyxl') as writer:
    # クラスタごとにシートを作成
    for cluster_id in sorted(result_df['cluster_id'].unique()):
        if cluster_id == -1:
            continue
        
        # クラスタに属するデータをフィルタリング
        cluster_df = result_df[result_df['cluster_id'] == cluster_id].copy()
        
        # distance_to_centerの値で降順にソート
        cluster_df = cluster_df.sort_values('distance_to_center', ascending=True)
        
        # オリジナルインデックスをリセット
        cluster_df = cluster_df.reset_index(drop=True)
        
        # Excelシートに書き込み
        cluster_df.to_excel(writer, sheet_name=f'Cluster_{cluster_id:04d}', index=False)
    
    # クラスタから漏れたデータを別シートに書き込み
    if -1 in clusters:
        outliers_df = result_df[result_df['cluster_id'] == -1].copy()
        outliers_df = outliers_df.sort_values('distance_to_center', ascending=False).reset_index(drop=True)
        outliers_df.to_excel(writer, sheet_name='Outliers', index=False)

print('Clustering results exported to clustering_results.xlsx with distance-based sorting')

Clustering results exported to clustering_results.xlsx with distance-based sorting
